# Modeling

In this notebook, we will be training different models to narrow down the best options.

In [1]:
# Importing libraries
import sys
import os

# To import script
sys.path.append('py_scripts')